In [2]:
import sys
sys.path.append('../notebooks')
import utils
import os
import numpy as np
import glob
import cv2
import utils
import pylab as pl
import shutil
import matplotlib.cm as cm
from sklearn.cross_validation import train_test_split
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/ipykernel/.*')
%matplotlib inline

In [10]:
os.environ['KERAS_BACKEND'] = 'theano'
# TODO: Use float16 ?
os.environ['THEANO_FLAGS'] = 'mode=FAST_RUN,device=gpu0,floatX=float16'
import keras
from keras.layers import Input, Dense, Dropout, Activation, Flatten, merge, RepeatVector, Permute, Reshape
from keras.layers import Convolution2D, MaxPooling2D, UpSampling2D, MaxoutDense
from keras.layers import LeakyReLU
from keras.models import Model
from keras.optimizers import SGD, RMSprop
from keras.utils.layer_utils import print_summary
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint

Using Theano backend.
Using gpu device 0: GeForce GTX 960 (CNMeM is disabled, cuDNN 4007)


In [11]:
import theano
print theano.config.device

gpu0


In [12]:
from keras.layers import Layer, InputSpec
import keras.initializations
import keras.backend as K

class Maxout2D(Layer):
    def __init__(self, output_dim, cardinality, init='glorot_uniform', **kwargs):
        super(Maxout2D, self).__init__(**kwargs)
        # the k of the maxout paper
        self.cardinality = cardinality
        # the m of the maxout paper
        self.output_dim = output_dim
        self.init = keras.initializations.get(init)
    
    def build(self, input_shape):
        self.input_dim = input_shape[1]
        self.input_spec = [InputSpec(dtype=K.floatx(),
                                     shape=(None, input_shape[1], input_shape[2], input_shape[3]))]
        self.W = self.init((self.input_dim, self.output_dim, self.cardinality),
                           name='{}_W'.format(self.name))
        self.b = K.zeros((self.output_dim, self.cardinality))
        self.trainable_weights = [self.W, self.b]
              
    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        # flatten the spatial dimensions
        flat_x = K.reshape(x, (-1, input_shape[1], input_shape[2] * input_shape[3]))
        output = K.dot(
            K.permute_dimensions(flat_x, (0, 2, 1)),
            K.permute_dimensions(self.W, (1, 0, 2))
        )
        output += K.reshape(self.b, (1, 1, self.output_dim, self.cardinality))
        output = K.max(output, axis=3)
        output = output.transpose(0, 2, 1)
        output = K.reshape(output, (-1, self.output_dim, input_shape[2], input_shape[3]))
        return output
    
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], self.output_dim, input_shape[2], input_shape[3])
    
    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'cardinality': self.cardinality
        }
        base_config = super(Maxout2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
input_img = Input(shape=(3, 16, 16))
x = Convolution2D(16, 3, 3, border_mode='same')(input_img)
x = Maxout2D(5, 7)(x)

model = Model(input=input_img, output=x)
model.compile(optimizer='adam', loss='mse')
print_summary(model.layers)

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
input_1 (InputLayer)               (None, 3, 16, 16)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)    (None, 16, 16, 16)  448         input_1[0][0]                    
____________________________________________________________________________________________________
maxout2d_1 (Maxout2D)              (None, 5, 16, 16)   595         convolution2d_1[0][0]            
Total params: 1043
____________________________________________________________________________________________________


    # maxout in numpy
    X = np.random.rand(10, 2, 3, 4)
    X = X.reshape(10, 2, 3 * 4) # 10, 3, 30

    # cardinality
    k = 2
    # output dim
    m = 7
    d = X.shape[1]

    #W = np.random.rand(d, m, k)
    W = np.ones((d, m, k), dtype=np.float32)
    for i in range(m):
        W[:,i,:] = i

    #X[0].T.dot(W[:,0,:]).shape

    print X.transpose(0, 2, 1).shape
    print W.transpose(1, 0, 2).shape

    out = np.dot(X.transpose(0, 2, 1), W.transpose(1, 0, 2))
    #out = np.tensordot(X.transpose(0, 2, 1), W.transpose(0, 1, 2), axes=1)
    out = np.max(out, axis=3)
    out = out.transpose(0, 2, 1)
    out = out.reshape(10, out.shape[1], 3, 4)
    print out.shape
    print out[0]

    k_W = K.variable(W)
    k_X = K.variable(X)

    flat_x = K.reshape(k_X, (-1, 2, 3 * 4))

    output = K.dot(
        K.permute_dimensions(flat_x, (0, 2, 1)),
        K.permute_dimensions(k_W, (1, 0, 2))
    )
    output = K.max(output, axis=3)
    output = output.transpose(0, 2, 1)
    output = K.reshape(output, (-1, W.shape[1], 3, 4))

    _f = K.function([], [output])
    k_out = _f([])[0]
    print k_out.shape
    print k_out